In [1]:
!pip install pandas datasets

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from datasets import load_dataset

train_dataset = load_dataset("LabHC/bias_in_bios", split='train').shuffle(seed=42).select(range(10000))
dev_dataset = load_dataset("LabHC/bias_in_bios", split='dev').shuffle(seed=42).select(range(1000))

/home/fc2456/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
train_dataset = load_dataset("LabHC/bias_in_bios", split='train')
test_dataset = load_dataset("LabHC/bias_in_bios", split = 'test')
dev_dataset = load_dataset("LabHC/bias_in_bios", split='dev')

In [ ]:
!pip install transformers

In [ ]:
!pip install torch torchvision torchaudio

In [7]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments
from transformers import AutoTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')

def tokenize(example):
    return tokenizer(example["hard_text"], padding="max_length", truncation=True, max_length=128)

train_dataset_tok = train_dataset.map(tokenize, batched = True)
dev_dataset_tok = dev_dataset.map(tokenize, batched = True)

train_dataset_tok = train_dataset_tok.rename_column('gender', 'labels')
dev_dataset_tok = dev_dataset_tok.rename_column('gender', 'labels')

train_dataset_tok.set_format("torch")
dev_dataset_tok.set_format("torch")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 39642/39642 [00:24<00:00, 1600.78 examples/s]


In [ ]:
!pip install transformers[torch]

In [3]:
from transformers import TrainingArguments, Trainer
import torch
from sklearn.metrics import accuracy_score
import numpy as np

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=3, 
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8, 
    warmup_steps=500, 
    warmup_ratio=0.01, 
    logging_dir='./results',
    logging_steps = 10
)

trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset=train_dataset_tok, 
    eval_dataset=  dev_dataset_tok
)


In [8]:
#full

trainer.train()

model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

Step,Training Loss
10,0.034800
20,0.000100
30,0.000100
40,0.000100
50,0.000100
60,0.000100
70,0.027700
80,0.000100
90,0.004500
100,0.030900


('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.json',
 './model/merges.txt',
 './model/added_tokens.json')

In [9]:
# full

dev_preds = trainer.predict(dev_dataset_tok)
pred_labels = np.argmax(dev_preds.predictions, axis = 1)
print('Validation Accuracy:', accuracy_score(dev_dataset['gender'], pred_labels))

Validation Accuracy: 0.9972251652287978


In [13]:
test_dataset_tok = test_dataset.map(tokenize, batched = True)

predictions = trainer.predict(test_dataset_tok)

test_dataset_with_preds = test_dataset_tok.add_column(
    "predictions", predictions.predictions.argmax(axis=-1)
)

Map: 100%|██████████| 99069/99069 [00:52<00:00, 1894.17 examples/s]


In [18]:
from datasets import Dataset
test_df = Dataset.to_pandas(test_dataset_with_preds)
profession_groups = test_df.groupby("profession")

for profession, group in profession_groups:
    gender_accuracy = (group["gender"] == group["predictions"]).mean()
    print(f"Profession {profession}: Gender Classification Accuracy = {gender_accuracy}")

Profession 0: Gender Classification Accuracy = 0.9907735982966643
Profession 1: Gender Classification Accuracy = 0.997626582278481
Profession 2: Gender Classification Accuracy = 0.9980351221908388
Profession 3: Gender Classification Accuracy = 0.9939759036144579
Profession 4: Gender Classification Accuracy = 0.9957325746799431
Profession 5: Gender Classification Accuracy = 0.9978586723768736
Profession 6: Gender Classification Accuracy = 0.9956128324650397
Profession 7: Gender Classification Accuracy = 0.9919110212335692
Profession 8: Gender Classification Accuracy = 0.9919354838709677
Profession 9: Gender Classification Accuracy = 0.996
Profession 10: Gender Classification Accuracy = 0.9972677595628415
Profession 11: Gender Classification Accuracy = 0.9961893301243482
Profession 12: Gender Classification Accuracy = 0.9935965848452508
Profession 13: Gender Classification Accuracy = 0.9970451667370198
Profession 14: Gender Classification Accuracy = 0.9948320413436692
Profession 15: Gend